<a href="https://colab.research.google.com/github/drfperez/DeepPurpose/blob/main/Tox21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd

# Read the CSV file
df = pd.read_csv('tox21.csv')

# Print the structure of the CSV file
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7831 entries, 0 to 7830
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   NR-AR          7265 non-null   float64
 1   NR-AR-LBD      6758 non-null   float64
 2   NR-AhR         6549 non-null   float64
 3   NR-Aromatase   5821 non-null   float64
 4   NR-ER          6193 non-null   float64
 5   NR-ER-LBD      6955 non-null   float64
 6   NR-PPAR-gamma  6450 non-null   float64
 7   SR-ARE         5832 non-null   float64
 8   SR-ATAD5       7072 non-null   float64
 9   SR-HSE         6467 non-null   float64
 10  SR-MMP         5810 non-null   float64
 11  SR-p53         6774 non-null   float64
 12  mol_id         7831 non-null   object 
 13  smiles         7831 non-null   object 
dtypes: float64(12), object(2)
memory usage: 856.6+ KB
None


In [ ]:

from google.colab import files
uploaded = files.upload()

# Read the uploaded CSV file
import pandas as pd
for f in uploaded:
    df = pd.read_csv(f)
    print(df)

Saving tox21.csv to tox21 (1).csv
      NR-AR  NR-AR-LBD  NR-AhR  NR-Aromatase  NR-ER  NR-ER-LBD  NR-PPAR-gamma  \
0       0.0        0.0     1.0           NaN    NaN        0.0            0.0   
1       0.0        0.0     0.0           0.0    0.0        0.0            0.0   
2       NaN        NaN     NaN           NaN    NaN        NaN            NaN   
3       0.0        0.0     0.0           0.0    0.0        0.0            0.0   
4       0.0        0.0     0.0           0.0    0.0        0.0            0.0   
...     ...        ...     ...           ...    ...        ...            ...   
7826    NaN        NaN     NaN           NaN    NaN        NaN            NaN   
7827    1.0        1.0     0.0           0.0    1.0        0.0            NaN   
7828    1.0        1.0     0.0           0.0    1.0        1.0            0.0   
7829    1.0        1.0     0.0           NaN    1.0        1.0            0.0   
7830    0.0        0.0     NaN           0.0    0.0        0.0            0

In [15]:
print(df.columns)

Index(['Formula', 'FW', 'DSSTox_CID', 'SR-HSE', 'ID', 'SMILES', 'NR-AR',
       'SR-ARE', 'NR-Aromatase', 'NR-ER-LBD', 'NR-AhR', 'SR-MMP', 'NR-ER',
       'NR-PPAR-gamma', 'SR-p53', 'SR-ATAD5', 'NR-AR-LBD', 'Compound ID',
       'Compound Batch ID', 'Sample ID', 'Stoichiometry', 'Alt ID',
       'PUBCHEM_SID'],
      dtype='object')


In [ ]:

import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Load the CSV file
df = pd.read_csv("tox21.csv")

# Drop rows with NaN values
df.dropna(inplace=True)

# Define the SMILES column and the toxicity label columns
smiles_col = 'smiles'
toxicity_cols = ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD',
                 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']

# Split the dataset into input features (X) and target labels (y)
X = df[smiles_col].values
y = df[toxicity_cols].values

# Convert SMILES to molecular descriptors using RDKit
def smiles_to_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    descriptors = np.array(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048))
    return descriptors

# Convert SMILES strings to descriptors
X_descriptors = np.array([smiles_to_descriptors(smiles) for smiles in X])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_descriptors, y, test_size=0.2, random_state=42)

# Train a random forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=toxicity_cols))

# Define a function to predict toxicity from a SMILES string
def predict_toxicity(smiles):
    descriptors = np.array([smiles_to_descriptors(smiles)])
    prediction = clf.predict(descriptors)[0]
    return prediction

# Test the function

smiles = 'CC1C(=O)OC2C1(C34C(=O)OC5C3(C2)C6(C(C5)C(C)(C)C)C(C(=O)OC6O4)O)O'  # example SMILES string
toxicity = predict_toxicity(smiles)
print(f"Toxicity prediction for {smiles}: {toxicity}")

Accuracy: 0.7662337662337663
Classification Report:
                precision    recall  f1-score   support

        NR-AR       0.40      0.15      0.22        13
    NR-AR-LBD       0.67      0.57      0.62         7
       NR-AhR       0.00      0.00      0.00        31
 NR-Aromatase       0.00      0.00      0.00        11
        NR-ER       0.57      0.08      0.14        50
    NR-ER-LBD       0.75      0.25      0.38        12
NR-PPAR-gamma       0.00      0.00      0.00         6
       SR-ARE       0.33      0.03      0.05        35
     SR-ATAD5       0.00      0.00      0.00         1
       SR-HSE       0.00      0.00      0.00         7
       SR-MMP       0.33      0.04      0.06        28
       SR-p53       0.00      0.00      0.00         2

    micro avg       0.47      0.07      0.13       203
    macro avg       0.25      0.09      0.12       203
 weighted avg       0.34      0.07      0.11       203
  samples avg       0.01      0.01      0.01       203

Toxicity 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:


!pip install rdkit

from rdkit import Chem
from rdkit.Chem import PandasTools

def sdf_to_csv(sdf_file, output_file, delimiter=','):
    """
    Convert an SDF file to a CSV or TSV file.

    Args:
        sdf_file (str): Path to the input SDF file.
        output_file (str): Path to the output CSV or TSV file.
        delimiter (str, optional): Delimiter to use in the output file. Default is ',' (CSV).
    """
    # Read the SDF file
    suppl = Chem.SDMolSupplier(sdf_file)

    # Convert to a pandas DataFrame
    df = PandasTools.LoadSDF(sdf_file, smilesName='SMILES', molColName='Molecule', includeFingerprints=False)

    # Drop the 'Molecule' column if present
    if 'Molecule' in df.columns:
        df = df.drop('Molecule', axis=1)

    # Save the DataFrame as CSV or TSV
    df.to_csv(output_file, sep=delimiter, index=False)

    print(f"Conversion complete! Output file: {output_file}")

# Example usage
sdf_file = 'tox21.sdf'  # Replace with the actual path to your SDF file
csv_file = 'output.csv'

sdf_to_csv(sdf_file, csv_file)

[15:42:33] Explicit valence for atom # 3 Cl, 2, is greater than permitted
[15:42:33] ERROR: Could not sanitize molecule ending on line 21572
[15:42:33] ERROR: Explicit valence for atom # 3 Cl, 2, is greater than permitted
[15:42:35] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 1 ignored.
[15:42:35] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 1 ignored.
[15:42:35] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 6 ignored.
[15:42:35] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 6 ignored.
[15:42:37] Explicit valence for atom # 2 Si, 8, is greater than permitted
[15:42:37] ERROR: Could not sanitize molecule ending on line 346021
[15:42:37] ERROR: Explicit valence for atom # 2 Si, 8, is greater than permitted
[15:42:38] Explicit valence for atom # 3 Cl, 2, is greater than permitted
[15:42:38] ERROR: Could not sanitize molecule ending on l

Conversion complete! Output file: output.csv
